<a href="https://colab.research.google.com/github/acouprie/vehicule-tour/blob/master/Routing_Optimization_Idai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing optimization in a flooding context

## Description of the project
In the context of the e-flooding project, the SEPIA team at IRIT is looking for a motivated student team working on the integration of a territory map and an algorithm of planning of civil security vehicles movement. This part of the project is called “Vehicle Tours” and should simulate emergency services interventions during a flood, which are deployed on the territory to save people, animals and goods.
## Content
The E-flooding project is a simulation of a flooding emergency services with constraints of resources such as vehicules (with a certain amount of places, an ability to drive only on roads, water or on airs) and the emergency stations, where they start from. Those saving point might change during the time if needed.
The vehicules needs to go at a precise location to save people in distress. Those saving zones are called challenge, in other words buildings where the emergency services need to have an intervention (hospitals, schools…).
The vehicles should opere a optimized circuit, such that they can pass through as much as challenge possible according to the following constraints: blocked roads, vehicle type and capacity.
## Goals
The goal of the product to be delivered is to implement an interaction between VROOM and QGIS, and adapting VROOM to run a custom algorithm and apply it to a predefined graph.
## Customer constraints
The project should be “Plug and Play”, that means can works with any map and algorithm without new configuration. The project should also be portable, that means can be executed easily on any computer, to achieve that, the solution picked is Docker in order to deliver the project in containers.
There is no limit in terms of financial spend into the wages and the numbers of hours worked.
## Technical specifications
The project will be developed using the programming language Python 3 which will interface VROOM tool with Openstreetmap and QGIS, a geographical information system.
The program should be able to take as an input a graph representing the different challenges.

We'll solve this complexe problem in this Python 3 Jupyter Notebook with [OpenRouteService](https://openrouteservice.org) new **route optimization service** which implement VROOM.

This Jupiter Notebook get its inspiration from the two awesome example projects of ORS [Disaster optimization](https://openrouteservice.org/disaster-optimization/) and [Avoided flooded areas](https://openrouteservice.org/example-avoid-flooded-areas-with-ors/).

## Requirements

You will need two csv input files, one for the vehicle fleet and one for the concerns. 

## The logistics setup

In total 20 sites were identified in need of public emergency services intervention, while 3 vehicles were scheduled for the operation.

The **vehicles** were all located in the Hôtel-Dieu at Paris, France and had the same following constraints:

- operation time windows from 8:00 to 20:00
- loading capacity:
  "car" : 4,
  "bus" : 50,
  "boat" : 10,
  "helicopter" : 3

The **concerns** are located in the Île de France region. Their needs range from 1 to 5 people to save and have a constraint for time window of the intervention (from 8:00 min to 20:00 max).

In [1]:
!pip install openrouteservice
!pip install fiona
!pip install pyproj

import folium
from folium.plugins import BeautifyIcon
import shapely
import pandas as pd
import openrouteservice as ors
from IPython.core.interactiveshell import InteractiveShell

import fiona as fn
from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import cascaded_union, transform

import pyproj
import time

     |████████████████████████████████| 14.7MB 263kB/s 
     |████████████████████████████████| 10.4MB 4.5MB/s 


In [2]:
!rm ./concerns_list*
!rm ./vehicle_list*
from google.colab import files
uploaded_csv = files.upload()

rm: cannot remove './concerns_list*': No such file or directory
rm: cannot remove './vehicle_list*': No such file or directory


Saving concerns_list.csv to concerns_list.csv
Saving vehicle_list.csv to vehicle_list.csv


In [0]:
# Insert ORS api key
api_key = '5b3ce3597851110001cf624855844106946f43c59ce6e46f048c554d' 
clnt = ors.Client(key=api_key)

## The routing problem setup

Now that we have described the setup sufficiently, we can start to set up our actual Vehicle Routing Problem. For this example we're using the FOSS library of [Vroom](https://github.com/VROOM-Project/vroom), which has [recently seen](http://k1z.blog.uni-heidelberg.de/2019/01/24/solve-routing-optimization-with-vroom-ors/) support for OpenRouteService and is available through their APIs.

To properly describe the problem in algorithmic terms, we have to provide the following information:

- **vehicles start/end address**: vehicle depot in Hôtel-Dieu at Paris, France
- **vehicle capacity**:  "car" : 4, "bus" : 50, "boat" : 10, "helicopter" : 3
- **vehicle operational times**: 08:00 - 20:00
- **service location**: pick-up location
- **service time windows**: individual pick-up location's time window
- **service amount**: individual pick-up location's needs

We defined all these parameters in the data sheets `concerns_list.csv` and `vehicle_list.csv`. Now we have to only wrap this informatoin into our code and send a request to OpenRouteService optimization service at [`https://api.openrouteservice.org/optimization`](https://openrouteservice.org/dev/#/api-docs/optimization/post).

In [0]:
# The vehicles are all located at Hospital Hôtel-Dieu at Paris, France
depot = [48.8539327,2.3477731]

#Define vehicle classes with respective names and capacities
class Vehicle :
  profile = "Vehicle"
  cap = 50
  start = list(reversed(depot))
  end=list(reversed(depot))
  time_window = [1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp

switcherC = {
  "car" : 4,
  "bus" : 50,
  "boat" : 10,
  "helicopter" : 3
}

In [0]:
# Get input vehicles from the file vehicle_list
vehicles_data = pd.read_csv('./vehicle_list.csv')

# Define the vehicles
vehicles = list()
for v in vehicles_data.itertuples() :
  vehicles.append (
    ors.optimization.Vehicle(
      id = v.ID, 
      start = Vehicle.start,
      end = Vehicle.end,
      capacity = [switcherC.get(v.Type,50)],
      time_window = Vehicle.time_window
    )
  )

In [0]:
#Function to create buffer around tweet point geometries and transform it to the needed coordinate system (WGS84)
def CreateBufferPolygon(point_in, resolution=2, radius=20):    
    sr_wgs = pyproj.Proj(init='epsg:4326') # WGS84
    sr_utm = pyproj.Proj(init='epsg:32632') # UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # Unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 20 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84
        
    return poly_wgs


# Function to request directions with avoided_polygon feature
def CreateRoute(avoided_point_list, coordinates):
    route_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False,
                    'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
    route_directions = clnt.directions(**route_request)
    
    return route_directions

In [7]:
# First define the map centered around Paris
m = folium.Map(location=[48.856697, 2.351462], tiles='cartodbpositron', zoom_start=11)

# Next define the concern places
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job

concerns_data = pd.read_csv(
    './concerns_list.csv',
    index_col="ID",
    parse_dates=["Save_From", "Save_Before"]
)

concerns = list()
for concern in concerns_data.itertuples():
    concerns.append(
        ors.optimization.Job(
            id=concern.Index,
            location=[concern.Lon, concern.Lat],
            service=1200,  # Assume 20 minutes at each site
            amount=[concern.Needed_Amount],
            time_windows=[[
                int(concern.Save_From.timestamp()),  # VROOM expects UNIX timestamp
                int(concern.Save_Before.timestamp())
            ]]
        )
    )

# Plot the locations on the map with more info in the ToolTip
for location in concerns_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Persons to save : <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))
    
    folium.Marker(
        location=[location.Lat, location.Lon],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)    

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="heart empty", prefix='fa')
).add_to(m)

# defining the flooded area

def style_function(color): # To style data
    return lambda feature: dict(color=color)

avoided_point_list =  []
flooded_area1 = Polygon(CreateBufferPolygon([2.345636,48.848296], resolution=5, radius=300))
avoided_point_list.append(flooded_area1) # Create flooded

folium.features.GeoJson(data=flooded_area1,
                        name='Flood affected areas',
                        style_function=style_function('#1d4591')).add_to(m)



/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [0]:
# Make the request
result = clnt.optimization(
    jobs=concerns,
    vehicles=vehicles,
    geometry=True
)

### Avoid flood areas
With a first set of generated routes, we use the pairs vehicle-concern to modify the routes to avoid the flood zones.

In [0]:
# Create a list to display the location of all concerns taken by each vehicule
paths = list()
for r in result['routes']:
    v = list()
    for step in r["steps"]:
      v.append(
        [
          step.get("job", "Safe Point"),  # Station ID
          step["location"]  # Location of the job
        ]
      )
    paths.append(v)

In [0]:
# Get coordinates
nb_paths = 0 
all_coordinates = list()
for v_path in paths :
  v_coordinates = list()
  for j in v_path:
    v_coordinates.append(j[1]) #j[0] = ID job  and j[1] = job location
    nb_paths = nb_paths + 1
  all_coordinates.append(v_coordinates)
nb_paths = (nb_paths/2) - 1


#for items in all_coordinates:
        #print(items)

#len(all_coordinates[1])

In [0]:
# Color generator to print the routes
import random
colors = list()
for _ in result['routes']:
  colors.append("#%06x" % random.randint(0, 0xFFFFFF))

#colors

In [12]:
for vehicle in range (0, len(all_coordinates)):
  for element in range (0, (len(all_coordinates[vehicle])-1)):
      try:
        coordinate = [all_coordinates[vehicle][element],all_coordinates[vehicle][element+1]] # coordinate has, for the vehicle 0, his start point (0) and the first point to arrive (1)
        route_directions = CreateRoute(avoided_point_list, coordinate)
        folium.features.GeoJson(data=route_directions, 
                                name='Alternative Route',
                                style_function=style_function(colors[vehicle]),
                                overlay=True,
                                tooltip=folium.Tooltip("<h4>Vehicle " + str(vehicle) + "</h4>")
                                ).add_to(m)
        time.sleep(2)
      except Exception: 
        print('Exception : could not create a new route')


# Check if flood affected tweet is located on route
m.add_child(folium.map.LayerControl())
m

In [13]:
# TODO TO DELETE WHEN EVERYONE AGREE
# Regular Route
for vehicle in range (0, len(all_coordinates)):
  for element in range (0, (len(all_coordinates[vehicle])-1)):
      try:
        print("starting point = ")
        print(all_coordinates[vehicle][element])
        print("ending point = ")
        print(all_coordinates[vehicle][element+1])
        coordinate = [all_coordinates[vehicle][element],all_coordinates[vehicle][element+1]] # coordinate has, for the vehicle 0, his start point (0) and the first point to arrive (1)
        route_directions = CreateRoute(avoided_point_list, coordinate)
        folium.features.GeoJson(data=route_directions, name='Alternative Route', style_function=style_function(colors[vehicle]), overlay=True).add_to(m)
        time.sleep(0.5)
      except Exception: 
        print('Exception : could not create a new route')

for items in all_coordinates:
        print(items)

len(all_coordinates[1])

# Check if flood affected tweet is located on route
m.add_child(folium.map.LayerControl())
m

starting point = 
[2.3477731, 48.8539327]
ending point = 
[2.3654542218261803, 48.8351795037258]
starting point = 
[2.3654542218261803, 48.8351795037258]
ending point = 
[2.3477731, 48.8539327]
starting point = 
[2.3477731, 48.8539327]
ending point = 
[2.34777310000021, 48.864327347405904]
starting point = 
[2.34777310000021, 48.864327347405904]
ending point = 
[2.31069424257813, 48.8778723546714]
starting point = 
[2.31069424257813, 48.8778723546714]
ending point = 
[2.3477731, 48.8539327]
starting point = 
[2.3477731, 48.8539327]
ending point = 
[2.3105225812011803, 48.889837887506]
starting point = 
[2.3105225812011803, 48.889837887506]
ending point = 
[2.32528545961915, 48.862290295651505]
starting point = 
[2.32528545961915, 48.862290295651505]
ending point = 
[2.34296658144532, 48.83020759960879]
starting point = 
[2.34296658144532, 48.83020759960879]
ending point = 
[2.40836956606446, 48.7996873840938]
starting point = 
[2.40836956606446, 48.7996873840938]
ending point = 
[2.532

/usr/local/lib/python3.6/dist-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  stacklevel=1)
/usr/local/lib/python3.6/dist-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  stacklevel=1)
/usr/local/lib/python3.6/dist-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  stacklevel=1)
/usr/local/lib/python3.6/dist-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  stacklevel=1)


starting point = 
[2.32871868715821, 48.969910582445]
ending point = 
[2.1663270245605504, 48.8225227772857]
starting point = 
[2.1663270245605504, 48.8225227772857]
ending point = 
[2.28528835878907, 48.8202623111443]
starting point = 
[2.28528835878907, 48.8202623111443]
ending point = 
[2.3170457135254, 48.83156362239121]
starting point = 
[2.3170457135254, 48.83156362239121]
ending point = 
[2.3477731, 48.8539327]
[[2.3477731, 48.8539327], [2.3654542218261803, 48.8351795037258], [2.3477731, 48.8539327]]
[[2.3477731, 48.8539327], [2.34777310000021, 48.864327347405904], [2.31069424257813, 48.8778723546714], [2.3477731, 48.8539327]]
[[2.3477731, 48.8539327], [2.3105225812011803, 48.889837887506], [2.32528545961915, 48.862290295651505], [2.34296658144532, 48.83020759960879], [2.40836956606446, 48.7996873840938], [2.5323090802246098, 48.692153711468], [2.5971970807129, 48.6336494994411], [2.6972756634765696, 48.6293383648808], [2.5498185406738303, 48.8907408304031], [2.4766907940918, 48

## Print the map with all informations


In [14]:
# TODO : Delete when Landry finish
# Add the output routes to the map
for color, route in zip(colors, result['routes']):
    decoded=ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature", 
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                        }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)

m

## Data view

In [15]:
# Only extract relevant fields from the response
extract_fields = ['Distance', 'Amount', 'Duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'Vehicle'
vehicles_df

KeyError: ignored

### Vehicle 0
Below you can observe the concerns reached by the vehicle 0. To change the vehicle you only need to pass in `paths[0]` the id of the wanted vehicle.

In [0]:
path_v1 = pd.DataFrame(paths[0], columns=["Station ID", "Location"])
path_v1

Alfaro Romero Sandra - Amiard Landry - Couprie Antoine <br />
University Paul Sabatier, Toulouse - France